# Convert h5 model to mlmodel

We need this conversion in order to pass our model to apple framework CoreML

source:
https://heartbeat.fritz.ai/using-coremltools-to-convert-a-keras-model-to-core-ml-for-ios-d4a0894d4aba


##First steps


1.   Import coereml tools using pip
2.   mount google drive in order to load models



In [ ]:
!pip install coremltools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import coremltools as ct
from sklearn.model_selection import train_test_split

## Load the .h5 model from drive

- Declare the output labels. In Covid_19 radiography case we have only 4 classes
    1.   COVID -> 0
    2.   Lung_Opacity -> 1
    3.   Normal -> 2
    4.   Viral Pneumonia -> 3


- load and cover model using coremltools.converters for keras 

In [ ]:
models_dir_path = "/content/drive/MyDrive/datasets/practical_ml/tensorFlow_models"
CORE_ML_MODELS_PATH = "/content/drive/MyDrive/datasets/practical_ml/coreML_models"
target_models = ["MobileNetV2", "DenseNet169", "VGG16"]
output_labels = ['0', '1', '2', '3']


In [ ]:
def signCoreModel(core_model):
  core_model.author = 'Athanasios Kottaridis'
  core_model.short_description = 'Pre trained MobileNerV2 NN for classifing lung Radiographies'
  # core_model.input_description['image'] = 'Takes as input a Radiography Image'
  # core_model.output_description['output'] = "Radiography Class"

  return core_model


In [ ]:
for model_name in target_models: 
  model_path = models_dir_path + "/" + model_name + ".h5"
  # Load model from path
  model = keras.models.load_model(model_path)
  # config classifier
  classifier_config = ct.ClassifierConfig(output_labels)
  # Convert to Core ML
  core_model = ct.convert(model, inputs=[ct.ImageType()], classifier_config=classifier_config)
  #sign model
  core_model = signCoreModel(core_model)
  # Store Model
  core_model.save('{}/{}.mlmodel'.format(CORE_ML_MODELS_PATH,model_name))

 

Translating MIL ==> MLModel Ops: 100%|██████████| 159/159 [00:04<00:00, 38.92 ops/s]


In [ ]:
model.summary()
# data = np.random.rand(1, 299, 299, 3)
# y_prob = model.predict_class(data) 
# y_prob
# y_classes = y_prob.argmax(axis=-1)
# y_classes 
# model.predict_classes

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               10617088  
_________________________________________________________________
dense_19 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 1028      
Total params: 25,398,596
Trainable params: 10,683,908
Non-trainable params: 14,714,688
_________________________________________________________________
